In [ ]:
"""
Notebook: Consumo_Eletrecidade.ipynb
Date: oct 2025

Description:
    Robust script for acquiring national daily electricity consumption (GWh)
from the National Energy Networks (REN) API.

Robustness features:
- Sequential extraction by date.
    - Progress control and visual feedback.
- Exception handling (including timeouts, HTTP errors, and missing data).
    - Cool-down mechanism (pauses) to mitigate API rate limiting (error 429).

Output File:
    target_consumption.csv (CSV file with ‘date’ and ‘consumption_gwh’ columns)

Requirements:
    pip install pandas requests
“”"

In [5]:
import requests
import pandas as pd
from datetime import date, timedelta
import time
import sys




REN_API_URL = "https://servicebus.ren.pt/datahubapi/electricity/ElectricityConsumptionSupplyDaily?culture=pt-PT&date={date}"


DATA_INICIO = date(2015, 1, 1)
DATA_FIM = date(2025, 9, 30) 



def get_dates_between(start_date, end_date):
    """Gera todas as datas entre as datas de início e fim."""
    delta = end_date - start_date
    return [start_date + timedelta(days=i) for i in range(delta.days + 1)]

def extract_daily_consumption(data_list):
    """Percorre as datas, faz chamadas à API e extrai o consumo total."""
    
    dados_finais = []
    
    print(f"A recolher dados para {len(data_list)} dias ({DATA_INICIO} a {DATA_FIM})...")
    
    for i, data_obj in enumerate(data_list):
        
        data_str = data_obj.strftime("%Y-%m-%d")
        url = REN_API_URL.format(date=data_str)
        
       
        sys.stdout.write(f"\rProgresso: {i + 1}/{len(data_list)} | Dia: {data_str}")
        sys.stdout.flush()

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            consumo_diario_list = response.json()
            
            
            gwh_value = None
            
            if isinstance(consumo_diario_list, list):
                
                for registo in consumo_diario_list:
                    if registo.get('type') == 'CONSUMO':
                        
                        gwh_value = registo.get('daily_Accumulation')
                        break # Encontrado, parar a procura
            
            # -----------------------------------------------------------
            
            if gwh_value is not None:
                dados_finais.append({
                    'date': data_str, 
                    'consumo_gwh': gwh_value
                })

        except requests.exceptions.HTTPError as err:
            
            if err.response.status_code == 429:
                print(f"\nAVISO: Limite de taxa atingido (429) em {data_str}. A pausar por 5 segundos...")
                time.sleep(5)
            
        except requests.exceptions.RequestException:
            
            pass
        except IndexError:
            
            pass
        except Exception as e:
            
            pass

        
        time.sleep(0.3) 

    return pd.DataFrame(dados_finais)


datas_a_recolher = get_dates_between(DATA_INICIO, DATA_FIM)
df_consumo = extract_daily_consumption(datas_a_recolher)

if not df_consumo.empty:
    df_consumo['date'] = pd.to_datetime(df_consumo['date'])
    df_consumo.sort_values(by='date', inplace=True)
    df_consumo.drop_duplicates(subset=['date'], keep='first', inplace=True)
    
    df_consumo.to_csv('target_consumo.csv', index=False)
    
    print("\n\n" + "-"*50)
    print("✅ SUCESSO! Ficheiro 'target_consumo.csv' criado automaticamente.")
    print(f"Total de registos únicos de consumo: {len(df_consumo)}")
    print("-" * 50)
else:
    print("\n\n❌ ERRO: Não foi possível extrair dados.")

A recolher dados para 3926 dias (2015-01-01 a 2025-09-30)...
Progresso: 3926/3926 | Dia: 2025-09-30

--------------------------------------------------
✅ SUCESSO! Ficheiro 'target_consumo.csv' criado automaticamente.
Total de registos únicos de consumo: 3926
--------------------------------------------------
